# Shortest Path
![title](img/ej1.png)

### EX3 Basic example SP with Dijkstra algorithm
1. Implement the Dijkstra algorithm and check the #EX1 solution

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

from mis_utils import *

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.latex.repr = True

# Model definitions 

In [2]:
# Definiciones de constantes

node_names = np.array(('s', '2', '3', '4', '5', 't'))

# Balances: Sale desde s y llega hasta t
beq = np.array([2, 0, 0, 0, 0, -2])

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# Matrices resultantes de NN a NA
Aeq, arc_idxs = nn2na(NN, node_names = node_names, show_results = True)

# Guardo los nombres de los nodo-arco-nodo posibles
nan_names = get_col_names(NN, node_names, as_numpy=True, sep = "->")

# Restricciones, l <= x <= u
# Entre 0 e infinito
# bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

[[0 1]
 [0 2]
 [1 3]
 [1 5]
 [2 4]
 [3 5]
 [4 5]]
Input: 
 [[0 1 1 0 0 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]

Column names: s2-s3-24-2t-35-4t-5t

Output: 
 [[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]]



In [3]:
# Costs vector
# According to column names: s2-s3-24-2t-35-4t-5t
C = np.array([2, 2, 2, 5, 2, 1, 2])

# Inputs to optimize

In [4]:
print("Node Node matrix:")
display(pd.DataFrame(data=NN, columns=node_names, index = node_names))

Node Node matrix:


,s,2,3,4,5,t
s,0,1,1,0,0,0
2,0,0,0,1,0,1
3,0,0,0,0,1,0
4,0,0,0,0,0,1
5,0,0,0,0,0,1
t,0,0,0,0,0,0


In [5]:
print("Node Arcs matrix")
pd.DataFrame(data=Aeq, columns=nan_names, index = node_names)

Node Arcs matrix


,s->2,s->3,2->4,2->t,3->5,4->t,5->t
s,1,1,0,0,0,0,0
2,-1,0,1,1,0,0,0
3,0,-1,0,0,1,0,0
4,0,0,-1,0,0,1,0
5,0,0,0,0,-1,0,1
t,0,0,0,-1,0,-1,-1


In [6]:
resume_df = pd.DataFrame(C, index=nan_names, columns=['Costs'])
# resume_df['Costs'] = C
resume_df.transpose()

,s->2,s->3,2->4,2->t,3->5,4->t,5->t
Costs,2,2,2,5,2,1,2


In [7]:
len_nodes = NN.shape[0]
initial_node = 0

# Solve trough Dijkstra algorithm

In [8]:
weights, prec = dijkstra(NN, C, node_names)


Sin explorar:  [0, 1, 2, 3, 4, 5]
Pesos de los nodos sin explorar:  [0.0, inf, inf, inf, inf, inf]
Nodo elegido  s
s  =>  2  con un costo de  2
s  =>  3  con un costo de  2
[ 0.  2.  2. inf inf inf]
[0 0 0 0 0 0]
Tabla de pesos:
 [[ 0.  2.  2. inf inf inf]
 [ 0.  2.  2. inf inf inf]]
Tabla de precedentes:
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]]


Sin explorar:  [1, 2, 3, 4, 5]
Pesos de los nodos sin explorar:  [2.0, 2.0, inf, inf, inf]
Nodo elegido  2
2  =>  4  con un costo de  2
2  =>  t  con un costo de  5
[ 0.  2.  2.  4. inf  7.]
[0 0 0 1 0 1]
Tabla de pesos:
 [[ 0.  2.  2. inf inf inf]
 [ 0.  2.  2. inf inf inf]
 [ 0.  2.  2.  4. inf  7.]]
Tabla de precedentes:
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 1 0 1]]


Sin explorar:  [2, 3, 4, 5]
Pesos de los nodos sin explorar:  [2.0, 4.0, inf, 7.0]
Nodo elegido  3
3  =>  5  con un costo de  2
[0. 2. 2. 4. 4. 7.]
[0 0 0 1 2 1]
Tabla de pesos:
 [[ 0.  2.  2. inf inf inf]
 [ 0.  2.  2. inf inf inf]
 [ 0.  2.  2.  4. inf  7.]
 [ 0.  2.  2.  4.  4.  

In [9]:
# Show results

prec_actual = np.argwhere(node_names == 't')[0][0]
prec_fin = np.argwhere(node_names == 's')[0][0]

fin = False

path = np.array(())
w_path = np.array(())
while not fin:
    path = np.append(node_names[prec_actual], path)
    w_path = np.append(weights[prec_actual], w_path)
    prec_actual = prec[prec_actual]

    if prec_actual == prec_fin:
        fin = True

path = np.append(node_names[prec_actual], path)
w_path = np.append(weights[prec_actual], w_path)

# Results

In [10]:
print("Best pat is from s to t is: ", path)
print("Accumulated costs from s to t are: ", w_path)

Best pat is from s to t is:  ['s' '2' '4' 't']
Accumulated costs from s to t are:  [0. 2. 4. 5.]


In [11]:
df = pd.DataFrame(w_path, index=path, columns = ['Accumulated cost']).transpose()
display(df)

,s,2,4,t
Accumulated cost,0.0,2.0,4.0,5.0


## Trough Dijkstra algorithm is possible to solve shortest path problem with O(n²) complexity

In [16]:
node_names

array(['s', '2', '3', '4', '5', 't'], dtype='<U1')